In [2]:
import urllib.request
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re

In [3]:
baseUrlPage = 'https://www.sbir.gov/sbirsearch/firm/all?firm=&uei=&city=&zip=&page='
baseUrl = 'https://www.sbir.gov/'
pageList = [i for i in range(1, 610)]

In [4]:
def filterHTMLstr(str):
    html_tag = {'&#xA;': '\n', '&quot;': '\"', '&amp;': '&', '&lt;': '<', '&gt;': '>',
                '&apos;': "'", '&nbsp;': ' ', '&yen;': '¥', '&copy;': '©', '&divide;': '÷'
        , '&times;': 'x', '&trade;': '™', '&reg;': '®', '&sect;': '§', '&euro;': '€',
                '&pound;': '£', '&cent;': '￠', '&raquo;': '»', '&nbsp': ' ', u'\xa0': ' ',
                '\n': '', '\t': ' ', '    ': '', '&emsp': ' ', '  ': ' ', '<br />': '\n'
                }
    for k, v in html_tag.items():
        str = str.replace(k, v)
        # str = str.replace(k[1:], v)
    str = str.strip('\n')
    str = str.strip(' ')

    return str

In [5]:
def createDict():
    dic = {"Company":"", 
               "Award Title":"", 
               "Agency":"", 
               "Branch":"", 
               "Phase":"", 
               "Program":"", 
               "Agency Tracking Number":"", 
               "Contract":"", 
               "Proposal Award Date":"", 
               "Contract End Date":"", 
               "Solicitation Number":"", 
               "Solicitation Year":"", 
               "Topic Code":"", 
               "Award Year":"", 
               "Award Amount":"", 
               "DUNS":"", 
               "Hubzone Owned":"", 
               "Socially and Economically Disadvantaged":"", 
               "Woman Owned":"", 
               "Number Employees":"", 
               "Company Website":"", 
               "Address1":"", 
               "Address2":"", 
               "City":"", 
               "State":"", 
               "Zip":"", 
               "Contact Name":"", 
               "Contact Title":"", 
               "Contact Phone":"", 
               "Contact Email":"", 
               "PI Name":"", 
               "PI Title":"", 
               "PI Phone":"", 
               "PI Email":"", 
               "RI Name":"", 
               "RI POC Name":"", 
               "RI POC Phone":"", 
               "Research Keywords":"", 
               "Abstract":""}
    return dic

In [6]:
def processSingle(detailPage, numberEmployee, companyName, totalData):
    awardBaseURL = "https://www.sbir.gov/sbirsearch/detail/"
    awardURLPostfix = re.findall(regAwardItem, detailPage)
    for k in awardURLPostfix:
        awardDetailRequest = urllib.request.urlopen(awardBaseURL + k[0])
        awardDetailPage = awardDetailRequest.read().decode('latin-1')
        dic = createDict()
        dic["Award Title"] = k[1]
        handleMix(dic, awardDetailPage)
        handleCompanyAddress(dic, awardDetailPage)
        handleContact(dic, awardDetailPage)
        handlePI(dic, awardDetailPage)
        handleRI(dic, awardDetailPage)
        handleAbstract(dic, awardDetailPage)
        handleWebsite(dic, detailPage)
        dic["Number Employees"] = numberEmployee
        dic["Company"] = companyName
        totalData.append(dic)

In [31]:
def to_CSV(totalData, counterCompany):
    fileName = 'task8-AdditionalSBC-InfoV2-' + str(int(counterCompany / 5000)) + '.csv'
    if counterCompany % 5000 == 0 or counterCompany == 50:
        pd.DataFrame(totalData).to_csv(fileName,index=False, encoding="utf-8")
    else:
        pd.DataFrame(totalData).to_csv(fileName,index=False, encoding="utf-8", header=False, mode='a')

In [32]:
regTable = re.compile(r'<table class="table table-striped">(.*?)</table>', re.S)
regItem = re.compile(r'<tr>(.*?)</tr>', re.S)
regCompanyNameAndLink = re.compile(r'<a href="(.*?)">(.*?)</a>')
regNumberOfAwards = re.compile(r'<td align="center">([0-9]*)</td>')
regNumEmployee = re.compile(r'<p><strong># of Employees:</strong>(.*?)</p>', re.S)
regHUBZone = re.compile(r'<p><strong>HUBZone Owned:</strong>(.*?)</p>', re.S)
regSocialDisadvantage = re.compile(r'<p><strong>Socially and Economically Disadvantaged:</strong>(.*?)</p>', re.S)
regWomanOwned = re.compile(r'<p><strong>Woman Owned:</strong>(.*?)</p>', re.S)
regWebsite = re.compile(r'a href="http://(.*?)" title="Company Website"', re.S)
regNxt = re.compile(r'<a href=\'.*?page=.*?\' data=\'(.*?)\'>', re.S)
regAwardItem = re.compile(r'<a href="/sbirsearch/detail/([0-9]*?)" title="(.*?)">', re.S)
regMix = re.compile(r'<span class="open-description">(.*?)</span>', re.S)

counterCompany = 0
counterRow = 0
fallBackCompany = 0
fallBackRow = 0
totalData = []
while counterCompany < 30419:
    try:
        for i in pageList:
            urlPage = baseUrlPage + str(i)
            a = urllib.request.urlopen(urlPage)
            html = a.read().decode('latin-1')
            table = re.findall(regTable, html)
            tr = re.findall(regItem, table[0])
            for j in range(len(tr)):
                nameAndLink = re.findall(regCompanyNameAndLink, tr[j])[0] #nameAndLink = ('<link-to-detail-page>', 'companyName')        
                urlDetail = baseUrl + nameAndLink[0]
                detailRequest = urllib.request.urlopen(urlDetail)
                detailPage = detailRequest.read().decode('latin-1')
                ### Number of Employee
                numberEmployee = re.findall(regNumEmployee, detailPage)[0].strip()

                ### Next
                nxt = re.findall(regNxt, detailPage)
                curNum = 0
                processSingle(detailPage, numberEmployee, nameAndLink[1], totalData)
                while len(nxt) > 0:
                    nxtNum = nxt[-1]
                    if int(nxtNum) > curNum:
                        urlNxt = urlDetail + "?page=" + nxtNum
                        curNum = int(nxtNum)
                    else:
                        break
                    detailRequest = urllib.request.urlopen(urlNxt)
                    detailPage = detailRequest.read().decode('latin-1')
                    nxt = re.findall(regNxt, detailPage)
                    processSingle(detailPage, numberEmployee, nameAndLink[1], totalData)
                counterCompany += 1
                if counterCompany % 50 == 0:
                    print(str(counterCompany) + " company finished" + ", page: " + str(int(counterCompany / 50)))
                    fallBackCompany = counterCompany
                    to_CSV(totalData, counterCompany)
                    totalData = []
    except Exception:
        print("Error at company number: " + str(counterCompany) + ", page " + str(int(counterCompany / 50)) + ". Retrying...")
        counterCompany = fallBackCompany
        pageList = [i for i in range(int(fallBackCompany / 50) + 1, 610)]
        totalData = []
print("Number of Company: " + str(counterCompany))

50 company finished, page: 1
100 company finished, page: 2
Error at company number: 100, page 2. Retrying...


KeyboardInterrupt: 

In [8]:
### Mix
def handleMix(dic, html):
    regMix = re.compile(r'<span class="open-description">(.*?)</span>', re.S)
    mix = re.findall(regMix, html)
    agency = mix[0]
    branch = mix[1]
    contract = mix[2]
    agencyTrackingNumber = mix[3]
    amount = mix[4]
    phase = mix[5]
    program = mix[6]
    solicitationTopicCode = mix[7]
    solicitationNumber = mix[8]
    solicitationYear = mix[9]
    awardYear = mix[10]
    awardStartDate = mix[11]
    awardEndDate = mix[12]
    DUNS = mix[13]
    HUBZone = mix[14]
    WomanOwned = mix[15]
    SociallyEcoDis = mix[16]
    dic["Agency"] = agency
    dic["Branch"] = branch
    dic["Contract"] = contract
    dic["Proposal Award Date"] = awardStartDate
    dic["Contract End Date"] = awardEndDate
    dic["Agency Tracking Number"] = agencyTrackingNumber
    dic["Award Amount"] = amount
    dic["Phase"] = phase
    dic["Program"] = program
    dic["Topic Code"] = solicitationTopicCode
    dic["Solicitation Number"] = solicitationNumber
    dic["Solicitation Year"] = solicitationYear
    dic["Award Year"] = awardYear
    dic["DUNS"] = DUNS
    dic["Hubzone Owned"] = HUBZone
    dic["Woman Owned"] = WomanOwned
    dic["Socially and Economically Disadvantaged"] = SociallyEcoDis


In [9]:
### Company Website
def handleWebsite(dic, html):
    regWebsite = re.compile(r'a href="http://(.*?)" title="Company Website"', re.S)
    website = re.findall(regWebsite, html)
    if len(website) > 0:
        website = "http://" + website[0]
        dic["Company Website"] = website


In [10]:
### Company Address
def handleCompanyAddress(dic, html):
    regCompanyAddress1 = re.compile(r'<span itemprop="streetAddress">(.*?)</span>', re.S)
    companyAddress1 = re.findall(regCompanyAddress1, html)
    if len(companyAddress1) > 0:
        companyAddress1 = companyAddress1[0]
        dic["Address1"] = companyAddress1

    regCompanyCity = re.compile(r'<span class="locality" itemprop="addressLocality">(.*?)</span>', re.S)
    companyCity = re.findall(regCompanyCity, html)
    if len(companyCity) > 0:
        companyCity = companyCity[0].lstrip()
        dic["City"] = companyCity

    regCompanyState = re.compile(r'<span class="region" itemprop="addressRegion">(.*?)</span>', re.S)
    companyState = re.findall(regCompanyState, html)
    if len(companyState) > 0:
        companyState = companyState[0].lstrip()
        dic["State"] = companyState

    regCompanyZip = re.compile(r'<span class="postal-code" itemprop="postalCode">(.*?)</span>', re.S)
    companyZip = re.findall(regCompanyZip, html)
    if len(companyZip) > 0:
        companyZip = companyZip[0].lstrip()
        dic["Zip"] = companyZip


In [11]:
### Contact Name
def handleContact(dic, html):
    regBusinessContactBlock = re.compile(r'<div class="award-sub">Business Contact</div>.*?<div class="award-sub-description">(.*?)</div>', re.S)
    businessContactBlock = re.findall(regBusinessContactBlock, html)[0]

    regBusinessContactName = re.compile(r'<span class="award-node-section-label">Name:</span>(.*?)<br />', re.S)
    businessContactName = re.findall(regBusinessContactName, businessContactBlock)
    if len(businessContactName) > 0:
        businessContactName = filterHTMLstr(businessContactName[0]).lstrip()
        dic["Contact Name"] = businessContactName

    regBusinessContactTitle = re.compile(r'Title:(.*?)<br />', re.S)
    businessContactTitle = re.findall(regBusinessContactTitle, businessContactBlock)
    if len(businessContactTitle) > 0:
        businessContactTitle = filterHTMLstr(businessContactTitle[0].lstrip())
        dic["Contact Title"] = businessContactTitle

    regBusinessContactPhone = re.compile(r'Phone:(.*?)<br />', re.S)
    businessContactPhone = re.findall(regBusinessContactPhone, businessContactBlock)
    if len(businessContactPhone) > 0:
        businessContactPhone = filterHTMLstr(businessContactPhone[0].lstrip())
        dic["Contact Phone"] = businessContactPhone

    regBusinessContactEmail = re.compile(r'Email:(.*?)<br />', re.S)
    businessContactEmail = re.findall(regBusinessContactEmail, businessContactBlock)
    if len(businessContactEmail) > 0:
        businessContactEmail = filterHTMLstr(businessContactEmail[0].lstrip())
        dic["Contact Email"] = businessContactPhone

In [12]:
### PI
def handlePI(dic, html):
    regPIBlock = re.compile(r'<div class="award-sub">Principal Investigator</div>.*?<div class="award-sub-description">(.*?)</div>', re.S)
    PIBlock = re.findall(regPIBlock, html)[0]

    regPIName = re.compile(r'<span class="award-node-section-label">Name:</span>(.*?)<br />', re.S)
    PIName = re.findall(regPIName, PIBlock)
    if len(PIName) > 0:
        PIName = filterHTMLstr(PIName[0]).lstrip()
        dic["PI Name"] = PIName

    regPITitle = re.compile(r'Title:</span>(.*?)<br />', re.S)
    PITitle = re.findall(regPITitle, PIBlock)
    if len(PITitle) > 0:
        PITitle = filterHTMLstr(PITitle[0].lstrip())
        dic["PI Title"] = PITitle

    regPIPhone = re.compile(r'Phone:</span>(.*?)<br />', re.S)
    PIPhone = re.findall(regPIPhone, PIBlock)
    if len(PIPhone) > 0:
        PIPhone = filterHTMLstr(PIPhone[0].lstrip())
        dic["PI Phone"] = PIPhone

    regPIEmail = re.compile(r'Email:</span>(.*?)<br />', re.S)
    PIEmail = re.findall(regPIEmail, PIBlock)
    if len(PIEmail) > 0:
        PIEmail = filterHTMLstr(PIEmail[0].lstrip())
        dic["PI Email"] = PIEmail

In [13]:
### RI
def handleRI(dic, html):
    regRIBlock = re.compile(r'<div class="award-sub">Research Institution</div>.*?<div class="award-sub-description">(.*)</div>', re.S)
    RIBlock = re.findall(regRIBlock, html)[0]

    regRIName = re.compile(r'<span class="award-node-section-label">Name:</span>(.*?)<br />', re.S)
    RIName = re.findall(regRIName, RIBlock)
    if len(RIName) > 0:
        RIName = filterHTMLstr(RIName[0]).lstrip()
        dic["RI Name"] = RIName

    regRIPOCName = re.compile(r'Contact:</span>(.*?)<br />', re.S)
    RIPOCName = re.findall(regRIPOCName, RIBlock)
    if len(RIPOCName) > 0:
        RIPOCName = filterHTMLstr(RIPOCName[0].lstrip())
        dic["RI POC Name"] = RIPOCName

    regRIPOCPhone = re.compile(r'Phone:</span>(.*?)<br />', re.S)
    RIPOCPhone = re.findall(regRIPOCPhone, RIBlock)
    if len(RIPOCPhone) > 0:
        RIPOCPhone = filterHTMLstr(RIPOCPhone[0].lstrip())
        dic["RI POC Phone"] = RIPOCPhone

In [14]:
### Abstract
def handleAbstract(dic, html):
    regAbstract = re.compile(r'<div class="info-wrapper">Abstract</div>.*?<p>(.*?)</p>.*?</div>', re.S)
    abstract = re.findall(regAbstract, html)[0]
    abstract = filterHTMLstr(abstract)
    dic["Abstract"] = abstract